# 麥克連大盤脈動_part_5_美元兌台幣匯率  
* 中央銀行匯率資料來源 [新臺幣/美元 銀行間收盤匯率](https://www.cbc.gov.tw/tw/lp-645-1-1-60.html)

In [1]:
import os
import sys
from pathlib import Path
from io import StringIO
import pandas as pd
import requests
import duckdb

In [ ]:
# 引用自建公用模組
sys.path.insert(0, str(Path.cwd().parent))
from proj_util_pkg.settings import ProjEnvSettings

## 公用參數設定

In [3]:
pd.set_option("display.max_columns", None)

In [4]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"

In [5]:
# # 設定資料庫路徑
# TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
# twstock_db_path = f"{os.getcwd()}\\data\\twstock.duckdb"

In [6]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

In [ ]:
# ipython 載入 sql 擴充, 並設置 duckdb 連接
%load_ext sql

%sql conn_duckdb

In [ ]:
%%sql
-- 方法1：查詢當前數據庫中的所有表
SHOW TABLES;

## 美元兌台幣匯率  

In [ ]:
def get_usd_twd_rate():
    """從中央銀行網站取得美元兌台幣匯率資料
    
    Returns:
        DataFrame: 包含日期和匯率的資料框
    """
    url = 'https://www.cbc.gov.tw/tw/lp-645-1-1-60.html'
    
    # 設定 User-Agent 來模擬瀏覽器請求
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        # 發送請求獲取網頁內容
        response = requests.get(url, headers=headers, verify=False)
        response.encoding = 'utf-8'  # 設定編碼
        
        # 使用 pandas 直接讀取 HTML 表格
        # 由於網頁中可能有多個表格,我們取第一個表格(index=0)
        # df = pd.read_html(response.text)[0]
        df = pd.read_html(StringIO(response.text))[0]
        
        # 重新命名欄位
        df.columns = ['date', 'rate']
        
        # 轉換日期格式
        df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
        
        return df
        
    except Exception as e:
        print(f"發生錯誤: {e}")
        return None

# 執行函數並顯示結果
usd_twd_df = get_usd_twd_rate()
usd_twd_df.head(3)

In [ ]:
usd_twd_df

In [11]:
# 欄位名稱rename
usd_twd_df.rename(
    columns={
        'date': 'Date', 
    }
    , inplace=True
)

In [ ]:
%%sql result <<

-- 使用窗口函數計算匯率差異
SELECT 
    t1."Date",
    t1.rate,
    t1.rate - LAG(t1.rate) OVER (ORDER BY t1."Date") AS rate_diff
FROM usd_twd_df t1
ORDER BY t1."Date" DESC;

In [ ]:
# 將 result 轉換為 DataFrame
us_tw_exchange_rate_df = result.DataFrame()

us_tw_exchange_rate_df.head(3)

In [ ]:
%%sql
-- 先检查表是否存在，如果不存在则创建
CREATE TABLE IF NOT EXISTS tw_sotck_barometer_part_05 (
    Date DATE PRIMARY KEY,
    美元兌台幣匯率 DOUBLE,
    美元兌台幣匯率_變動 DOUBLE,
    美元兌台幣匯率_多空分數 INTEGER
);

-- 使用UPSERT方式插入数据
INSERT OR REPLACE INTO tw_sotck_barometer_part_05
SELECT 
    CAST(date AS DATE) AS Date,
    rate AS 美元兌台幣匯率,
    ROUND(rate_diff, 3) AS 美元兌台幣匯率_變動,
    CASE 
        WHEN 美元兌台幣匯率_變動 <= -0.1 THEN 1    -- 台币强势（升值）
        WHEN 美元兌台幣匯率_變動 >= 0.1 THEN -1    -- 台币弱势（贬值）
        ELSE 0                                  -- 其他情况不计分
    END AS 美元兌台幣匯率_多空分數
FROM us_tw_exchange_rate_df;

In [ ]:
%%sql
SELECT * FROM tw_sotck_barometer_part_05 ORDER BY Date DESC LIMIT 10;

In [16]:
# 關閉資料庫連線
conn_duckdb.close()